<a href="https://colab.research.google.com/github/markste-in/collatz/blob/main/create_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ray &> /dev/null
!pip install "ray[default]" &> /dev/null

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [24]:
import pandas as pd
import numpy as np
import sympy
from sympy.ntheory import factorint
from multiprocessing import Pool
import psutil
import time
import os

In [25]:
print("CPU Count:", psutil.cpu_count())

CPU Count: 2


In [26]:
pool = Pool()

In [27]:
def collatz_serie(n : int) -> int:
    ret = [n]
    while (n>1):
        if n%2 == 0: 
            _n = int(n / 2)
        else: 
            _n = n*3+1
        n = _n
        ret.append(n)
    return ret

In [28]:
collatz_serie(19)

[19, 58, 29, 88, 44, 22, 11, 34, 17, 52, 26, 13, 40, 20, 10, 5, 16, 8, 4, 2, 1]

In [29]:
xheader = ["Number", 
          "Length",
          "Max", 
          "IsEven",
          "nPrimes",
          "Smallest_Prime",
          "nSmallest_Prim",
          "Biggest_Prime",
          "nBiggest_Prime", 
          "isPrime"]

def all_fancy_calculations(serie):
        calculations = list()
        num_in_question = serie[0]
        fac_n = factorint(num_in_question)
        
        calculations.append(num_in_question)       # Starting number
        calculations.append(len(serie))     # Length of series
        calculations.append(max(serie))     # Max of number series
        calculations.append(num_in_question%2 ==0) # Is even?
        calculations.append(len(fac_n)) # how many primes
        calculations.append(min(fac_n)) # smallest prime
        calculations.append(fac_n[min(fac_n)]) # how often does the smallest prime appear
        calculations.append(max(fac_n)) # biggest prime
        calculations.append(fac_n[max(fac_n)]) # how often does the biggest prime appear
        calculations.append(sympy.isprime(num_in_question)) #is prime?
        
        return calculations
    

In [30]:
SMALLEST_NUMBER = 3 # since we calculate the previous serie the smallest number is actually one smaller (should not go below 2 since the return value of the collatz could be an empty list)

def create_dataset(highest_number):
    cz_series = pool.map(collatz_serie, range(3,highest_number))
    dataset = pool.map(all_fancy_calculations, cz_series)

    return pd.DataFrame(dataset)

In [31]:
%%time
highest_number = int(1e5)

df = create_dataset(highest_number)
df.columns = xheader

CPU times: user 4.51 s, sys: 696 ms, total: 5.21 s
Wall time: 14.5 s


In [32]:
df.sample(5)

,Number,Length,Max,IsEven,nPrimes,Smallest_Prime,nSmallest_Prim,Biggest_Prime,nBiggest_Prime,isPrime
21987,21990,101,70468,True,4,2,1,733,1,False
77892,77895,232,12141952,False,3,3,3,577,1,False
52100,52103,79,351700,False,1,52103,1,52103,1,True
98038,98041,67,372256,False,1,98041,1,98041,1,True
54286,54289,92,162868,False,1,233,2,233,2,False


In [33]:
# Save dataframe
PATH = "gdrive/MyDrive/collatz_dbs/"
os.makedirs(PATH,exist_ok=True)
FILENAME = 'collatz_db_' + time.strftime("%Y%m%d-%H%M%S") + '.parquet'
file = os.path.join(PATH,FILENAME)
df.to_parquet(file)